In [2]:
from nilearn import image, plotting, datasets, masking
from nilearn.input_data import NiftiLabelsMasker
from nilearn.signal import clean
from matplotlib import pyplot as plt 
from tqdm import tqdm
from nilearn.connectome import ConnectivityMeasure
from scipy.signal import correlate, correlation_lags
from scipy.stats import mode, pearsonr, spearmanr, chi2_contingency, chisquare
import pandas as pd
import numpy as np 
import math
import networkx as nx
from math import log10
from statsmodels.tsa.stattools import grangercausalitytests
from glob import glob 
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report

%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [15]:
def load_time_series(fname, gsr=False, to_csv=None):
    img = image.load_img(fname) 

#     ref = type('',(object,),\
#        {"maps": 'data/templates_GMatlas_GMatlas.nii.gz',\
#         "labels":[line.strip() for line in open('data/templates_GMatlas_GMatlas.txt').readlines()]})()

    ref = type('',(object,),\
       {"maps": 'data/46_rois.nii.gz',\
        "labels":[line.strip() for line in open('data/46_rois.txt').readlines()]})()

    parcellation = image.load_img(ref.maps)
    masker = NiftiLabelsMasker(labels_img=parcellation, standardize=True,\
                               memory='nilearn_cache', high_pass=0.01, low_pass=0.1, t_r=0.735)
    time_series = masker.fit_transform(img)
    
    if gsr:
        gs = np.mean(time_series, axis=1)
        time_series = clean(time_series, confounds=[gs]);
        
    time_series = pd.DataFrame(time_series)
    time_series.columns = ref.labels
    time_series.index = np.array(range(1, len(time_series)+1))*0.735
    
    if to_csv!=None:
        time_series.to_csv(to_csv)
        
    return time_series

In [1]:
def load_frames(fname, time_series):
    with open(fname, encoding='utf-16') as f:
        content = f.read() 
 
    header, content = content.split('*** Header End ***') 
    frames = content.split('*** LogFrame Start ***') 
    dicts = [] 
    for frame in frames[1:-1]: 
       lines = frame.split('\n') 
       line_dict = {} 
       for line in lines[1:-3]: 
          k, v = line.split(': ') 
          line_dict[k] = v 
       dicts.append(line_dict) 
    
    frames = pd.DataFrame(dicts) 
    frames['time_ms'] = frames.apply(lambda x: x['ExperimenterWindow.OnsetTime']\
          if math.isnan(float(x['ExperimenterWindow.OnsetTime']))==False else x['EightSecFix.OnsetTime'], axis=1)
    frames['time_sec'] = ((frames['time_ms'].astype(int)/1000))
    frames['time_sec'] = frames['time_sec']-frames['time_sec'].min()+0.735
    
    frames_cats = frames[frames['Procedure']!='TrialsPROC']
    ys = np.array([frames_cats.loc[frames_cats['time_sec']<=idx, 'Procedure'].iloc[-1].replace('PromptPROC', '')\
          for idx in time_series.index])
    
    return frames, ys

In [ ]:
from nilearn.plotting import view_connectome, view_markers
from nilearn.image import coord_transform

def get_mni_coords(img_fname, labels):
    img = image.load_img(img_fname)
    mni = datasets.load_mni152_brain_mask()

    img_resampled = image.resample_to_img(img, mni, interpolation='nearest')

    img_data = img_resampled.get_fdata()
    rois = set(img_data.flatten())

    mean_coord = {}
    for roi in rois:
        if roi==0:
            continue
        x, y, z =  np.mean(np.where(img_data==roi), axis=1)
        x, y, z = coord_transform(x, y, z, affine=mni.affine)
        mean_coord[roi] = [x, y, z]
    mean_coord = pd.DataFrame(mean_coord).transpose()
    mean_coord.columns = ['x','y', 'z']
    mean_coord.index = labels
    return mean_coord

In [14]:
def create_stat_map(conns):
    atlas = image.load_img(ref.maps)
    circuit = atlas.get_fdata().copy()
    rois  = ref.labels
    for roi_idx in range(1, len(rois)+1):
        roi = rois[roi_idx-1]
        circuit[circuit==roi_idx] = conns[roi] #conns[roi] if conns[roi]==1 else (0.5 if conns[roi] >0.78 else 0)
    return image.new_img_like(atlas, circuit)

In [17]:
def window_timeseries(time_series, window, ys, kind='covariance'):
    
    rois         = time_series.columns

    chunks       = [time_series.iloc[ts:ts+window] for ts in range(len(time_series))[::window] if len(time_series.iloc[ts:ts+window])==window]
    window_ys    = [mode(ys[ts:ts+window])[0][0] for ts in range(len(time_series))[::window]   if len(time_series.iloc[ts:ts+window])==window]
    window_times = [int(np.mean(chunk.index)) for chunk in chunks]

    conn         = ConnectivityMeasure(kind=kind)
    corrs        = conn.fit_transform([chunk.values for chunk in chunks])
    corrs        = [pd.DataFrame(corr, index=rois, columns=rois) for corr in corrs]
    
    return chunks, corrs, window_ys, window_times

In [3]:
def compute_crosscorrs_cov(timeseries, zerolag=True):
    rois       = timeseries.columns
    lags       = correlation_lags(len(timeseries), len(timeseries))
    crosscorrs = timeseries.corr(lambda x, y: np.max(abs(correlate(x, y))))
    crosslags  = timeseries.corr(lambda x, y: lags[np.argmax(abs(correlate(x,y)))])
    
    crosslags = pd.DataFrame(index=rois, columns=rois)
    for roi_x in timeseries:
        for roi_y in timeseries:
            if zerolag:
                crosslags.loc[roi_x, roi_y] =\
                    lags[np.argmax(abs(correlate(timeseries[roi_x],timeseries[roi_y])))]
            else:
                lags_sorted = lags[np.argsort(abs(correlate(timeseries[roi_x],timeseries[roi_y])))]
                crosslags.loc[roi_x, roi_y] = lags_sorted[-1] if lags_sorted[-1]!=0 else lags_sorted[-2]
    
    covs       = pd.DataFrame(np.cov(timeseries.transpose()), columns=timeseries.columns, index=timeseries.columns)
    #pearson    = timeseries.corr(lambda x, y: pearsonr(x, y)[0])
    #pvals      = timeseries.corr(lambda x, y: pearsonr(x, y)[1])

    return crosscorrs, crosslags, covs#pearson, pvals

In [ ]:
def compute_tvfcn_crosscorrs_covs_full(chunks, zerolag=True):
    X, Y         = chunks[0].columns, chunks[0].columns
    tvfcn_edges  = ['%s > %s' % (x, y) for x in X for y in Y]
    
    corrs, lags, covs = [], [], []
    for chunk in chunks:
        a, b, c = compute_crosscorrs_cov(chunk, zerolag)
        corrs.append(a.values.flatten())
        lags.append(b.values.flatten())
        covs.append(c.values.flatten())
        
    corrs, lags, covs  = pd.DataFrame(corrs).transpose(), pd.DataFrame(lags).transpose(), \
                         pd.DataFrame(covs).transpose()
    corrs.index = lags.index = covs.index = tvfcn_edges
    covs = covs.round(1)
    return corrs, lags, covs

In [ ]:
def compute_tvfcn_crosscorrs_covs(chunks):
    indices      = np.tril_indices(chunks[0].shape[1], k=-1)
    X, Y         = chunks[0].columns[indices[0]], chunks[0].columns[indices[1]]
    tvfcn_edges  = ['%s > %s' % (x, y) for (x, y) in zip(X, Y)]
    
    corrs, lags, covs = [], [], []
    for chunk in tqdm(chunks):
        a, b, c = compute_crosscorrs_cov(chunk)
        corrs.append(a.values[indices])
        lags.append(b.values[indices])
        covs.append(c.values[indices])
        
    corrs, lags, covs  = pd.DataFrame(corrs).transpose(), pd.DataFrame(lags).transpose(), \
                         pd.DataFrame(covs).transpose()
    corrs.index = lags.index = covs.index = tvfcn_edges
    return corrs, lags, covs

In [31]:
def compute_crosscorrs_forloop(timeseries):
    rois = timeseries.columns
    results = []
    
    lags = correlation_lags(len(timeseries), len(timeseries))
        
    for roi_x in rois:
        for roi_y in rois:
            crosscorrs  = correlate(timeseries[roi_x], timeseries[roi_y])
            maxcross    = np.max(abs(crosscorrs))
            maxcorr_lag = lags[np.argmax(abs(crosscorrs))]
            pearson, pval = pearsonr(timeseries[roi_x], np.roll(timeseries[roi_y], maxcorr_lag))
            results.append((roi_x, roi_y, maxcross, maxcorr_lag, pearson, pval))
        
    results = pd.DataFrame(results)
    results.columns = ['roi_x', 'roi_y', 'cross', 'lag', 'pearsonr', 'pval']

    cross   = results.pivot(index='roi_x', columns='roi_y', values='cross')
    lags    = results.pivot(index='roi_x', columns='roi_y', values='lag')
    pearson = results.pivot(index='roi_x', columns='roi_y', values='pearsonr')
    pvals   = results.pivot(index='roi_x', columns='roi_y', values='pval')

    return cross, lags, pearson, pvals

In [ ]:
def compute_tvfcn_crosscorrs(chunks):
    indices      = np.tril_indices(chunks[0].shape[1], k=-1)
    X, Y         = chunks[0].columns[indices[0]], chunks[0].columns[indices[1]]
    tvfcn_edges  = ['%s > %s' % (x, y) for (x, y) in zip(X, Y)]
    
    corrs, lags, pearson, pvals = [], [], [], []
    for chunk in chunks:
        a, b, c, d = compute_crosscorrs_forloop(chunk)
        corrs.append(a.values[indices])
        lags.append(b.values[indices])
        pearson.append(c.values[indices])
        pvals.append(d.values[indices])
        
    corrs, lags, pearson, pvals =   pd.DataFrame(corrs).transpose(),   pd.DataFrame(lags).transpose(), \
                                    pd.DataFrame(pearson).transpose(), pd.DataFrame(pvals).transpose()
    corrs.index = lags.index = pearson.index = pvals.index = tvfcn_edges
    return corrs, lags, pearson, pvals

In [4]:
from numpy.linalg import norm

def compute_instabilities_from_ts(time_series, win_len=30, nonoverlapping=True):
    chunks = []
    steps = range(len(time_series))
    steps = steps[::win_len] if nonoverlapping else steps
    for step in steps:
        chunk = time_series[step:step+win_len]
        if len(chunk)<win_len:
            continue
        chunks.append(chunk.values)

    conn = ConnectivityMeasure(kind='correlation')
    corrs = conn.fit_transform(chunks)

    instabilities = []
    for idx in range(1, len(corrs)):
        instabilities.append(norm(corrs[idx]-corrs[idx-1])/corrs[0].shape[0])
    
    return instabilities

def compute_instabilities(corrs):
     return np.array([norm(corrs[idx]-corrs[idx-1]) for idx in range(1, len(corrs))])/np.prod(corrs[0].shape)

In [23]:
def compute_tvfcn(corrs):
    indices      = np.tril_indices(len(corrs[0]), k=-1)

    tvfcn        = []
    for corr in corrs:
        tvfcn.append(corr.values[indices])

    X, Y         = corrs[0].columns[indices[0]], corrs[0].columns[indices[1]]
    tvfcn_edges  = ['%s > %s' % (x, y) for (x, y) in zip(X, Y)]
    tvfcn = pd.DataFrame(tvfcn).transpose()
    tvfcn.index = tvfcn_edges
    return tvfcn

In [24]:
from scipy.cluster.hierarchy import linkage, optimal_leaf_ordering, leaves_list, fcluster, dendrogram

def reorder_columns_by_dist(dist_matrix):
    linkage_matrix  = linkage(dist_matrix, method='average')
    ordered_linkage = optimal_leaf_ordering(linkage_matrix, dist_matrix)
    index           = leaves_list(ordered_linkage)
    cols            = [dist_matrix.index[idx] for idx in index]
    return cols

In [ ]:
def cluster_matrix(matrix, method='average', thresh=0.7, toplot=False, roi_to_highlight='Amygdala', metric='euclidean'):
    rois            = np.array(matrix.index)
    linkage_matrix  = linkage(matrix, method=method, metric=metric)
    ordered_linkage = optimal_leaf_ordering(linkage_matrix, matrix)
    index           = leaves_list(ordered_linkage)
    reorderedcols   = [rois[idx] for idx in index]
    
    color_thresh    = thresh = thresh*max(linkage_matrix[:,2])

    clusters        = fcluster(linkage_matrix, t=color_thresh, criterion='distance')
    
    dn = None
    if toplot:
        dn = dendrogram(linkage_matrix, labels=list(rois), orientation='right', leaf_font_size=10)#, color_threshold=thresh)        
        [x.set_color("red") for x in plt.gca().get_yticklabels() if (x.get_text()==roi_to_highlight)];
        
    return linkage_matrix, color_thresh, reorderedcols, clusters, dn

In [ ]:
def reorder_edges_tvfcn(edges, rois, clusters):
    edge_order = []
    for edge in tqdm(edges):
        a, b      = edge.split(' > ')
        cluster_a = clusters[rois==a][0]
        cluster_b = clusters[rois==b][0]
        edge_order.append((edge, cluster_a if (cluster_a==cluster_b) else cluster_a+0.5))

    edge_order    = pd.DataFrame(edge_order).sort_values(1)
    #edges_ordered = edge_order[0].values, edge_order[1].values

    return edge_order[0].values, edge_order[1].values 

In [ ]:
from scipy.stats import ttest_rel, ttest_ind
def compute_stimulus_pval_edges(edge, stimulus, method=ttest_rel, alternative='greater'):
    face_edge = edge[np.array(stimulus)=='Face']
    shape_edge = edge[np.array(stimulus)=='Shape']

    min_len = min(len(face_edge), len(shape_edge))
    return method(face_edge[:min_len], shape_edge[:min_len], alternative=alternative)[1]

In [ ]:
def get_edge_types(covs, lags, rois, window_ys):
    indices = np.tril_indices(len(rois), k=-1)
    X, Y = indices

    results = pd.DataFrame()

    rel_dict = {0:'-', 1:'x <> y', -1: 'x <<>> y', 2:'x > y', -2: 'x >> y', 3: 'x < y', -3:'x << y'}

    for x, y in zip(X, Y):
        covs_xy = covs.loc['%s > %s' % (rois[x], rois[y])]#rois[x]+' > '+rois[y]
        lags_xy = lags.loc['%s > %s' % (rois[x], rois[y])]#rois[x]+' > '+rois[y]
        rel = pd.DataFrame(pd.concat([covs_xy, lags_xy], axis=1))
        rel.columns = ['cov', 'lag']
        rel.loc[rel['cov']==0, 'rel'] = 0
        rel.loc[(rel['cov']>0) & (rel['lag']==0), 'rel'] = 1
        rel.loc[(rel['cov']<0) & (rel['lag']==0), 'rel'] = -1
        rel.loc[(rel['cov']>0) & (rel['lag']<0), 'rel'] = 2
        rel.loc[(rel['cov']<0) & (rel['lag']<0), 'rel'] = -2
        rel.loc[(rel['cov']>0) & (rel['lag']>0), 'rel'] = 3
        rel.loc[(rel['cov']<0) & (rel['lag']>0), 'rel'] = -3
        results["%s, %s" % (rois[x], rois[y])] = rel['rel']
    # results.replace(rel_dict)
    # print(results)
    results = results.transpose()
    results.columns   = window_ys
    return results, rel_dict

In [ ]:
def count_edge_types(edge_types, rel_dict):
    counts = edge_types.apply(lambda x: pd.Series([round(100*sum(x==edge_type)/len(x)) for edge_type in range(-3, 4)]), axis=1)
    #counts = edge_types.apply(lambda x: pd.Series([sum(x==edge_type) for edge_type in range(-3, 4)]), axis=1)
    counts.columns = [rel_dict[x] for x in range(-3, 4)]
    counts = counts[['-', 'x > y', 'x >> y', 'x < y','x << y' ]]# 'x <> y', 'x <<>> y',
    #counts = counts.replace({0:1})
    return counts

In [ ]:
def edge_labels(x):
    nonzeros = x[x.index!='-'][x>0].index
    if len(nonzeros)<1:
        return None, None
    elif len(nonzeros)==1:
        arrowhead = (nonzeros[0].split(' ')[1])
        if arrowhead=='>':
            return 'excitatory', '>'
        elif arrowhead=='<':
            return 'excitatory', '<'
        elif arrowhead=='>>':
            return 'inhibitory', '>'
        elif arrowhead=='<<':
            return 'inhibitory', '<'
    else:
        a, b = nonzeros[0].split(' ')[1], nonzeros[1].split(' ')[1]
        if   len(set(['>', '<']).intersection(set([a, b])))==2:
            return '+ve feedback loop', None
        elif len(set(['>', '<<']).intersection(set([a, b])))==2:
            return '-ve feedback loop', None
        elif len(set(['>>', '<<']).intersection(set([a, b])))==2:
            return '+ve feedback loop', None
        elif len(set(['>>', '<']).intersection(set([a, b])))==2:
            return '-ve feedback loop', None
    return None, None

In [ ]:
def construct_graph(edges):
    G = nx.DiGraph()
    for rois, (nature, arrow, pval) in edges.iterrows():
        a, b = rois.split(', ')
        w = -log10(pval)
        if nature==None:
            continue
        if 'loop' in nature:
            G.add_edge(a, b, label=nature, weight=w)
            G.add_edge(b, a, label=nature, weight=w)
        elif arrow=='>':
            G.add_edge(a, b, label=nature, weight=w)
        elif arrow=='<':
            G.add_edge(b, a, label=nature, weight=w)
    return G

# Plotting

## Plot windowed timeseries

In [22]:
from random import randint

def plot_windowed_timeseries(chunks, ys, x_indices):
    # Plot time series
    plt.figure(figsize=(20, 5))
    plt.xticks(x_indices[::2], ys[::2], rotation=90);
    [plt.plot(chunk, linewidth=3, color='#%06X' % randint(0, 0xFFFFFF)) for chunk in chunks];
    #print(set([len(chunk) for chunk in chunks]));
    plt.xlabel('Stimulus');
    plt.xlabel('BOLD');

## Plot windowed matrices

In [19]:
def plot_windowed_matrices(corrs, window_ys):
    # Plot matrices
    fig, axs = plt.subplots(1, len(corrs), figsize=(20, 4));
    for idx, corr in enumerate(corrs):
        axs[idx].imshow(corr, cmap='bwr', vmin=-1, vmax=1);
        axs[idx].set_xticks([]);
        axs[idx].set_yticks([]);
        axs[idx].set_xlabel(window_ys[idx], rotation=90);

In [1]:
def plot_tvfcn(data, xticks, reorder=True, figsize=(20, 4), vmin=None, vmax=None, cmap='bwr', fontsize=10, xlabel=None, ylabel=None, xrotation=90):
    if reorder:
        data = data.loc[reorder_columns_by_dist(data)]

    plt.figure(figsize=figsize);
    plt.imshow(data, aspect='auto', cmap=cmap, vmin=vmin, vmax=vmax);
    plt.colorbar();
    plt.xticks(range(len(xticks)), xticks, rotation=xrotation, fontsize=fontsize);
    plt.xlabel(xlabel);
    plt.ylabel(ylabel);
    plt.grid();

In [ ]:
def draw_graph(G, layout=None, pos=None, edge_weights=False, figsize=(20, 10), xlim=None, ylim=None, edgecolor='blue', fontcolor='red'):
    plt.figure(figsize=figsize)
    if pos==None:
        pos = layout(G)

    weights = [G[u][v]['weight'] for u,v in G.edges()] if edge_weights else None
    nx.draw(G, with_labels=True, pos=pos, edge_color=edgecolor, arrowsize=50, font_size=20, node_size=0, width=weights)

    edge_labels_formatted = {(e[0], e[1]):G.get_edge_data(e[0], e[1])['label'] for e in list(G.edges())}
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels_formatted, font_color=fontcolor, font_size=10);
    
    # plt.ylim(-50, 70);
    # plt.xlim(-110, 60);

# Granger Causality

In [3]:
def compute_gc(chunk):
    rois    = chunk.columns
    gclags  = pd.DataFrame(index=rois, columns=rois)
    gcstats = pd.DataFrame(index=rois, columns=rois)
    gcpvals = pd.DataFrame(index=rois, columns=rois)
    corrs   = pd.DataFrame(index=rois, columns=rois)
    for roi_x in chunk:
        for roi_y in chunk:
            chunk_xy  = np.array([chunk[roi_y], chunk[roi_x]]).transpose();
            result_xy = gcresult_to_df(grangercausalitytests(chunk_xy, maxlag=2, verbose=False))
            lag, pval = result_xy['ssr_ftest_pval'].idxmin(), result_xy['ssr_ftest_pval'].min()
            stat      = result_xy.loc[lag, 'ssr_ftest_stat']
            gclags.loc[roi_x, roi_y]  = lag
            gcpvals.loc[roi_x, roi_y] = pval
            gcstats.loc[roi_x, roi_y] = stat
            corrs.loc[roi_x, roi_y]   = pearsonr(chunk[roi_y], chunk[roi_x])[0]
    return gcstats, gclags, gcpvals, corrs

def compute_tvfcn_gc(chunks, window=10):
    X, Y         = chunks[0].columns, chunks[0].columns
    tvfcn_edges  = ['%s > %s' % (x, y) for x in X for y in Y]
    stats, lags, pvals, corrs = [], [], [], []
    for chunk in tqdm(chunks):
        if len(chunk)<window:
            continue
        a, b, c, d = compute_gc(chunk)
        stats.append(a.values.flatten())
        lags.append(b.values.flatten())
        pvals.append(c.values.flatten())
        corrs.append(d.values.flatten())
    stats, lags, pvals, corrs  = \
        pd.DataFrame(stats).transpose(),\
        pd.DataFrame(lags).transpose(), \
        pd.DataFrame(pvals).transpose(),\
        pd.DataFrame(corrs).transpose()

    stats.index = lags.index = pvals.index = corrs.index = tvfcn_edges
    return stats, lags, pvals, corrs

def get_edge_types_gc(stats, lags, pvals, corrs, rois, window_ys):
    indices = np.tril_indices(len(rois), k=-1)
    X, Y = indices
    results = pd.DataFrame()
    rel_dict = {0:'-', 1:'x <> y', -1: 'x <<>> y', 2:'x > y', -2: 'x >> y', 3: 'x < y', -3:'x << y'}
    for x, y in zip(X, Y):
        
        xy_pval = pvals.loc[rois[x]+' > '+rois[y]] 
        yx_pval = pvals.loc[rois[y]+' > '+rois[x]]
        
        xy_corr = corrs.loc[rois[x]+' > '+rois[y]]
        yx_corr = corrs.loc[rois[y]+' > '+rois[x]]
        
        rel = pd.DataFrame(pd.concat([xy_pval, yx_pval, xy_corr, yx_corr], axis=1))
        rel.columns = ['xy_pval', 'yx_pval', 'xy_corr', 'yx_corr']
        rel['rel'] = None
        rel.loc[(rel['xy_pval']>rel['yx_pval']) & (rel['xy_corr']>0), 'rel'] = 'x < y'
        rel.loc[(rel['xy_pval']>rel['yx_pval']) & (rel['xy_corr']<0), 'rel'] = 'x << y'
        rel.loc[(rel['xy_pval']<rel['yx_pval']) & (rel['xy_corr']>0), 'rel'] = 'x > y'
        rel.loc[(rel['xy_pval']<rel['yx_pval']) & (rel['xy_corr']<0), 'rel'] = 'x >> y'
        rel.loc[(rel['xy_pval']>(0.001/np.prod(pvals.shape))) &\
                (rel['yx_pval']>(0.001/np.prod(pvals.shape))), 'rel'] = '-'
        
        results["%s, %s" % (rois[x], rois[y])] = rel['rel']
    results = results.transpose()
    results.columns   = window_ys#[:-1]
    rel_dict = {rel_dict[k]:k for k in rel_dict}
    return results, rel_dict

# def gcresult_to_df(result):
#     lags = result.keys()
#     tests = result[key][0].keys()
#     b = []
#     for lag in lags:
#         tmp = []
#         for test in tests:
#             tmp = tmp + list(result[lag][0][test][:2])
#         b.append(tmp)

#     df = pd.DataFrame(b)
#     df.columns = np.concatenate([[test+'_stat', test+'_pval']for test in tests])
#     df.index = lags
#     return df

def gcresult_to_df(result):
    lags = result.keys()
    tests = result[1][0].keys()
    b = []
    for lag in lags:
        tmp = []
        for test in tests:
            tmp = tmp + list(result[lag][0][test][:2])+[result[lag][1][0].params[-1], result[lag][1][1].params[-1]]
        b.append(tmp)

    df = pd.DataFrame(b)
    df.columns = list(np.concatenate([[test+'_stat', test+'_pval', test+'_r1', test+'_r2'] for test in tests]))
    df.index = lags
    return df

In [1]:
from scipy.optimize import curve_fit

def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)

def fit_sigmoid(ydata, xdata):
    p0 = [max(ydata), np.median(xdata),1,min(ydata)] # this is an mandatory initial guess
    popt, pcov = curve_fit(sigmoid, xdata, ydata,p0, method='dogbox')
    
    return popt, pcov

def get_sigmoid(y, x):
    popt, pcov = fit_sigmoid(y, x)
    fitted = sigmoid(x, *popt)
    return x, fitted

In [ ]:
# Saving all instabilities

def save_instabilities_to_disk(inpath, outpath, subfile='data/lcn_subjects.csv'):
    fnames = glob(inpath+'*.csv')

    all_instabilities = []
    for fname in tqdm(fnames):
        all_instabilities.append([fname]+list(compute_instabilities_from_ts(pd.read_csv(fname, header=None))))
    df = pd.DataFrame(all_instabilities)

    df['subject'] = df[0].apply(lambda x: int(x.split('/')[-1].split('_')[0].split('-')[1]))
    df['ses']     = df[0].apply(lambda x: x.split('/')[-1].split('_')[1].split('-')[1])
    df['task']    = df[0].apply(lambda x: x.split('/')[-1].split('_')[2].split('-')[1])
    df['run']     = df[0].apply(lambda x: int(x.split('/')[-1].split('_')[3].split('-')[1].replace('.csv', '')))

    subjs = pd.read_csv(subfile)
    merged = pd.merge(df, subjs[['subject', 'age']], on='subject')
    merged.to_csv(outpath)

In [4]:
def coarsen(source_img, dest_img):
    resampled   = image.resample_to_img(source_img, dest_img, 'nearest')

    dest_data   = dest_img.get_fdata()
    labels      = set(dest_data.flatten()).difference({0})

    masks       = {label: image.new_img_like(\
                    ref_niimg=dest_img,\
                    data=dest_data==label,\
                    affine=dest_img.affine) for label in labels}

    result_data = dest_data.copy()
    total       = abs(resampled.get_fdata()).sum()
    activations = []

    for label in labels:
        masked 	= masking.apply_mask(resampled, masks[label])
        #1 normalize by total activation of source image
#         val = np.sum(abs(masked))/total 
        #2 normalize by number of voxels in the dest image
        val = np.mean(masking.apply_mask(resampled, masks[label])) 
        
        activations.append(val)
        result_data[result_data==label] = val
    
    result_img    = image.new_img_like(dest_img, result_data, dest_img.affine)
    
    results       = pd.Series(activations)
    results.index = labels
    results.index = results.index.astype(int)
    return results, result_img

In [4]:
def compute_instabilities_from_ts_subnetworks(time_series, fname, rois, nonoverlapping=True, win_len=30):
    chunks = []
    steps = range(len(time_series))
    steps = steps[::win_len] if nonoverlapping else steps
    for step in steps:
        chunk = time_series[step:step+win_len]
        if len(chunk)<win_len:
            continue
        chunks.append(chunk.values)

    conn = ConnectivityMeasure(kind='correlation')
    corrs = conn.fit_transform(chunks)
    corrs = [pd.DataFrame(corr) for corr in corrs]

    networks = rois['functional_network'].unique()
    instabilities = []
    for network in networks:
        idcs = np.where(rois['functional_network']==network)[0]
        tau1 = [norm(corrs[idx].loc[idcs, idcs]-corrs[idx-1].loc[idcs, idcs])/len(idcs)\
                         for idx in range(1, len(corrs))]
        instabilities.append([fname, network]+tau1)
    return instabilities

def save_instabilities_to_disk_subnetworks(fnames, outpath, rois, subfile='data/lcn_subjects.csv'):

    all_instabilities = []
    for fname in tqdm(fnames):
        ts = pd.read_csv(fname, header=None)
        all_instabilities = all_instabilities + \
            compute_instabilities_from_ts_subnetworks(ts, fname, rois=rois)
    df = pd.DataFrame(all_instabilities)
    
    df['subject'] = df[0].apply(lambda x: int(x.split('/')[-1].split('_')[0].split('-')[1]))
    df['ses']     = df[0].apply(lambda x: x.split('/')[-1].split('_')[1].split('-')[1])
    df['task']    = df[0].apply(lambda x: x.split('/')[-1].split('_')[2].split('-')[1])
    df['run']     = df[0].apply(lambda x: int(x.split('/')[-1].split('_')[3].split('-')[1].replace('.csv', '')))

    subjs = pd.read_csv(subfile)
    merged = pd.merge(df, subjs[['subject', 'age']], on='subject')
    merged.to_csv(outpath)
    
    
def save_instabilities_to_disk_subnetworks_biobank(fnames, outpath, rois, subfile='out/age_diabetes.csv'):
    all_instabilities = []
    for fname in tqdm(fnames):
        ts = pd.read_csv(fname, header=None)
        all_instabilities = all_instabilities + \
            compute_instabilities_from_ts_subnetworks(ts, fname, rois=rois)
    df = pd.DataFrame(all_instabilities)
    df.index = df[0].apply(lambda x: int(x.split('/')[-1].replace('.csv', '')))
    subjs = pd.read_csv(subfile, index_col=0)
    merged = pd.merge(df, subjs, left_index=True, right_index=True)
    merged.to_csv(outpath)

In [ ]:
def set_fontsize(small=8, medium=10, large=12):
    plt.rc('font', size=small)          # controls default text sizes
    plt.rc('axes', titlesize=small)     # fontsize of the axes title
    plt.rc('axes', labelsize=medium)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=small)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=small)    # fontsize of the tick labels
    plt.rc('legend', fontsize=small)    # legend fontsize
    plt.rc('figure', titlesize=large)  # fontsize of the figure title
    
def sigmoid_plot(toplot, color='black', desc=None, fontsize=20, ax=None, fit_sigmoid=True):
        
    ax.tick_params(axis='x', which='both', rotation=90)
    ax.tick_params(axis='y', which='both')
    
    ax.errorbar(toplot.index, y=toplot['mean'], yerr=toplot['sem'], marker='o', fmt='.', color=color);
    ax.set_ylabel('Instability');
    ax.set_xlabel('Age in YEARS');
    ax.set_xticks(toplot.index);
    ax.set_xticklabels(["%s (n=%s)" % (idx, int(x['n'])) for idx, x in toplot.iterrows()]);
    if desc is not None:
        ax.set_title('%s (N=%s)' % (desc, int(toplot['n'].sum())))

    if fit_sigmoid:
        x, y = get_sigmoid(toplot['mean'].values, toplot.index)
        ax.plot(x, y, color=color, linewidth=2);